## <div align="center"> <h2 align="center"> РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ </h1> </div>
## <div align="center"> <h3 align="center"> Факультет физико-математических и естественных наук </h2> </div>
## <div align="center"> <h3 align="center"> Кафедра математического моделирования и искусственного интеллекта </h2> </div>
## <div align="center"> <h3 align="center"> Дисциплина: МОЗИиИБ </h2> </div>
## Отчет по лабораторной работе № 5
## Студент: Евдокимов Максим Михайлович
## Группа: НФИмд-01-24
## вероятностные алгоритмы проверки чисел на простоту

# Задание

Реализовать все представленные аглоритмы проверки чисел на простоту.

In [124]:
# Создание списка для теста
tests = Matrix(undef, 0, 2)
for j in 1:10
    p = rand(3:10000)
    k = rand(1:p-1)
    tests = vcat(tests, [p, k]')
end
tests

10×2 Matrix{Any}:
 2851   550
 7759  3312
 6263  4214
 4902  2564
 1393   210
 1078   290
 1310   371
 8561  6590
 3508  1684
 8999  3569

In [125]:
# 1. Алгоритм теста Ферма
function fermat_test(n::Int, k::Int=5)
    if n <= 1
        return false
    elseif n <= 3
        return true
    end

    for _ in 1:k
        a = rand(2:n-2)
        if powermod(a, n-1, n) != 1
            return false
        end
    end
    return true
end

println("Тест Ферма:")
for i in 1:size(tests)[1]
    t = fermat_test(tests[i, 1], tests[i, 2])
    println("Число ", tests[i, 1], t ? "" : " НЕ", " простое.")
end

Тест Ферма:
Число 2851 простое.
Число 7759 простое.
Число 6263 простое.
Число 4902 НЕ простое.
Число 1393 НЕ простое.
Число 1078 НЕ простое.
Число 1310 НЕ простое.
Число 8561 НЕ простое.
Число 3508 НЕ простое.
Число 8999 простое.


In [130]:
# 2. Алгоритм вычисления символа Якоби
function jacobi_symbol(n::Int, a::Int=5)
    if n < 0 || iseven(n)
        return "Не подходит n четное."
    end
    a %= n
    g = 1
    
    while a != 0
        while iseven(a)
            a >>= 1
            if (n % 8) in [3, 5]
                g = -g
            end
        end
        a, n = n, a
        if a % 4 == 3 && n % 4 == 3
            g = -g
        end
        a %= n
    end
    return n == 1 ? g : 0
end

println("Символ Якоби:")
for i in 1:size(tests)[1]
    t = jacobi_symbol(tests[i, 1])
    println("При n = ", tests[i, 1], " и a = ", tests[i, 2], " символ Якоби = ", t)
end

Символ Якоби:
При n = 2851 и a = 550 символ Якоби = 1
При n = 7759 и a = 3312 символ Якоби = 1
При n = 6263 и a = 4214 символ Якоби = -1
При n = 4902 и a = 2564 символ Якоби = Не подходит n четное.
При n = 1393 и a = 210 символ Якоби = -1
При n = 1078 и a = 290 символ Якоби = Не подходит n четное.
При n = 1310 и a = 371 символ Якоби = Не подходит n четное.
При n = 8561 и a = 6590 символ Якоби = 1
При n = 3508 и a = 1684 символ Якоби = Не подходит n четное.
При n = 8999 и a = 3569 символ Якоби = 1


In [129]:
# 3. Алгоритм теста Соловэя-Штрассена
function solovay_strassen__test(n::Int, k::Int=5)
    if n <= 1
        return false
    elseif n <= 3
        return true
    end

    for _ in 1:k
        a = rand(2:n-2)
        x = jacobi_symbol(n, a)
        if typeof(x) == String
            return x
        end
        if x == 0 || powermod(a, (n-1) ÷ 2, n) != x % n
            return false
        end
    end
    return true
end

println("Тест Соловэя-Штрассена:")
for i in 1:size(tests)[1]
    t = solovay_strassen__test(tests[i, 1], tests[i, 2])
    println("При n = ", tests[i, 1], " и k = ", tests[i, 2], ": ", t)
end

Тест Соловэя-Штрассена:
При n = 2851 и k = 550: false
При n = 7759 и k = 3312: false
При n = 6263 и k = 4214: false
При n = 4902 и k = 2564: Не подходит n четное.
При n = 1393 и k = 210: false
При n = 1078 и k = 290: Не подходит n четное.
При n = 1310 и k = 371: Не подходит n четное.
При n = 8561 и k = 6590: false
При n = 3508 и k = 1684: Не подходит n четное.
При n = 8999 и k = 3569: false


In [131]:
# 4. Алгоритм теста Миллера-Рабина
function miller_rabin__test(n::Int, k::Int=5)
    if n <= 1
        return false
    elseif n <= 3
        return true
    elseif iseven(n)
        return false
    end

    # Разложение n-1 на d * 2^r
    r, d = 0, n - 1
    while iseven(d)
        d >>= 1
        r += 1
    end

    for _ in 1:k
        a = rand(2:n-2)
        x = powermod(a, d, n)
        if x == 1 || x == n - 1
            continue
        end

        for _ in 1:(r-1)
            x = powermod(x, 2, n)
            if x == n - 1
                break
            end
        end

        if x != n - 1
            return false
        end
    end
    return true
end

println("Тест Миллера-Рабина:")
for i in 1:size(tests)[1]
    t = miller_rabin__test(tests[i, 1], tests[i, 2])
    println("Число ", tests[i, 1], t ? "" : " НЕ", " простое.")
end

Тест Миллера-Рабина:
Число 2851 простое.
Число 7759 простое.
Число 6263 простое.
Число 4902 НЕ простое.
Число 1393 НЕ простое.
Число 1078 НЕ простое.
Число 1310 НЕ простое.
Число 8561 НЕ простое.
Число 3508 НЕ простое.
Число 8999 простое.
